In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import numpy as np
import seaborn as sns
import pyparetomixture.fit as pm
import pyparetomixture.type as pmt
from loguru import logger
import sys

logger.remove()
logger.add(sys.stdout, colorize=True, level="DEBUG")
None


In [28]:
# generate data
n = 10000
p = 0.95
alpha = 2.0
beta = 3.0  # beta > alpha
seed = 42
np.random.seed(42)


def generate_data(n, p, alpha, beta):
    x = np.random.binomial(1, p, n)
    rv_alpha = np.random.pareto(alpha, n)
    rv_beta = np.random.pareto(beta, n)
    return x * rv_alpha + (1 - x) * rv_beta


data = generate_data(n, p, alpha, beta)
logger.debug(
    f"Generated a sample of length {len(data)}. Mean: {np.mean(data)}, std: {np.std(data)}"
)


2023-04-05 21:48:42.163 | DEBUG    | __main__:<module>:18 - Generated a sample of length 10000. Mean: 1.0045454558001314, std: 2.6453820059904203


In [29]:
def pot(full_sample, k):
    full_sample.sort()
    return (full_sample[-k:] / full_sample[-k - 1]).copy()


def hill(sample):
    # hill estimate
    k = len(sample)
    assert (sample >= 1).all(), "not a pot sample"
    return k / (np.sum(np.log(sample)))


sample = pot(data, 200)
logger.debug(f"Generated pot sample. Min: {np.min(sample)}, max: {np.max(sample)}")


2023-04-05 21:48:42.648 | DEBUG    | __main__:<module>:14 - Generated pot sample. Min: 1.0012025510790947, max: 18.068859723443744


In [30]:
def backtracking_line_search(
    sample, pmp, direction, p_greater_than_1, pre_normalize=True
):
    # backtracking line search
    # https://en.wikipedia.org/wiki/Backtracking_line_search

    old_ll = pm.loglikelihood(pmp, sample)
    logger.debug(
        f"Doing backtracking line search in direction {direction}. Old ll: {old_ll}"
    )

    if pre_normalize:
        direction = direction * (1 / np.linalg.norm(np.array([direction.dalpha, direction.dbeta, direction.dp])))
        logger.debug(f"We normalize our direction: {direction}")

    multiplier = 0.5
    n_iterations = 100
    for i in range(n_iterations):
        new_pmp = pmp + direction * multiplier

        logger.debug(f"New pmp in backtracking line search: {new_pmp}")
        # we ensure our new pmp do not surpass the thresholds we are given
        if not p_greater_than_1 and new_pmp.p >= 1:
            logger.debug(f"Encountered p >= 1 in {pmp} where p should be < 1")
            new_ll = -np.inf
        elif p_greater_than_1 and new_pmp.p <= 1:
            logger.debug(f"Encountered p <= 1 in {pmp} where p should be > 1")
            new_ll = -np.inf
        else:
            new_ll = pm.loglikelihood(new_pmp, sample)

        if new_ll == np.nan:
            logger.debug(f"Encountered np.nan ll with params {pmp}")
            new_ll = -np.inf

        logger.debug(f"Its likelihood is {new_ll}")

        if new_ll > old_ll:
            logger.debug(f"Found direction: {direction * multiplier}; new ll: {new_ll}")
            return direction * multiplier
        elif i > (n_iterations - 20) and new_ll > old_ll - 1e-5:
            logger.debug("Making a step for the sake of it.")
            return direction * multiplier

        multiplier *= 0.8
    raise Exception(
        f"backtracking line search did not converge in {n_iterations}, old_ll={old_ll}, new_ll={new_ll}, multiplier={multiplier}, direction={direction}"
    )


def search(sample, p_greater_than_1):
    if not p_greater_than_1:
        # p smaller than 1
        # alpha < hill estimate
        # D > 0
        # bias hill > 0
        # need a beta > alpha
        alpha = hill(sample)
        beta = alpha
        p = 1

        # theta = np.array([alpha, beta, p])
        # direction = np.array([1, 0, -1])
        pmp = pmt.ParetoMixtureParameters(alpha, beta, p)
        logger.debug(pmp)
        direction = pmt.DeltaParetoMixtureParameters(-1, 0, -1)

        direction = backtracking_line_search(sample, pmp, direction, p_greater_than_1, pre_normalize = False)
        pmp = pmp + direction 
        old_ll = pm.loglikelihood(pmp, sample)
        
        n_iterations = 100
        for _ in range(n_iterations):
            logger.debug("-" * 80)
            logger.debug(f"Got new pmp: {pmp}")
            direction = pm.gradient(pmp, sample)
            direction = pmt.DeltaParetoMixtureParameters(
                direction.dll_dalpha, direction.dll_dbeta, direction.dll_dp
            )
            direction = backtracking_line_search(
                sample, pmp, direction, p_greater_than_1, pre_normalize=False
            )
            pmp = pmp + direction
            new_ll = pm.loglikelihood(pmp, sample)
            if (new_ll - old_ll) < 1e-6:
                logger.debug(f"Got final pmp: {pmp}")
                return pmp
            else:
                logger.debug(f"old_ll={old_ll}, new_ll={new_ll}")
            old_ll = new_ll
        raise Exception(f"Did not finish in {n_iterations} iterations")


pmp = search(sample, False)
pmp

2023-04-05 21:48:43.370 | DEBUG    | __main__:search:65 - ParetoMixtureParameters(alpha=1.76774613964902, beta=1.76774613964902, p=1)
2023-04-05 21:48:43.371 | DEBUG    | __main__:backtracking_line_search:8 - Doing backtracking line search in direction DeltaParetoMixtureParameters(dalpha=-1, dbeta=0, dp=-1). Old ll: -313.1384170578415
2023-04-05 21:48:43.371 | DEBUG    | __main__:backtracking_line_search:21 - New pmp in backtracking line search: ParetoMixtureParameters(alpha=1.26774613964902, beta=1.76774613964902, p=0.5)
2023-04-05 21:48:43.372 | DEBUG    | __main__:backtracking_line_search:36 - Its likelihood is -318.9198371887315
2023-04-05 21:48:43.372 | DEBUG    | __main__:backtracking_line_search:21 - New pmp in backtracking line search: ParetoMixtureParameters(alpha=1.3677461396490198, beta=1.76774613964902, p=0.6)
2023-04-05 21:48:43.373 | DEBUG    | __main__:backtracking_line_search:36 - Its likelihood is -315.3484228881297
2023-04-05 21:48:43.373 | DEBUG    | __main__:backtra

/home/maurits/private/pareto_mixture/src/pyparetomixture/fit.py:92: RuntimeWarning: invalid value encountered in log
  dll_dalpha2 = np.sum(


ParetoMixtureParameters(alpha=1.557758371763343, beta=1.7329990175955965, p=0.9646678798893905)

In [31]:
pm.gradient(pmp, sample)

Gradient(dll_dalpha=12.770953667566388, dll_dbeta=0.06882616343819181, dll_dp=-7.925597363783659)

Mathematics suggests we should move into the direction of our gradient. However, log-likelihood decreases when we do this, for any step size. 

1. Is our gradient correct?
2. Could floating point errors be the cause?

Next steps:
1. Create plot of ll surface, see if there's a "hole"
2. Plot steps as well?
3. Use higher precision floats
4. Random step sizes, simulated annealing, maybe..
5. Plot alpha, beta, because p is apparently "correct".

Look at log likelihood, see which parts can take extra log. Then things become a subtraction

log(log(likelihood))
Or:
sqrt(log(likelihood)), to make sure log(likelihood) is not close to 0

Or normal likelihood, but this wasnot convex.